# Лабораторная работа №2

## Задания

Решить задачу классификации датасета MNIST используя MLP из scikitlearn и используя CNN (по типу LeNet) c пакетом PyTorch. Сравнить результаты по метрикам, сделать обоснованные выводы

## Реализация

### Библиотеки

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score

### Датасет

In [2]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
y = y.astype(np.int64)  # т.к. там классификация цифр, то лучше если брать по числовому значению
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)  # stratify нужен, чтобы все классы были так или иначе включены

In [3]:
def one_hot_transform(y, classes_count):  # преобразование в матрицу вхождений
    transformed = np.zeros((len(y), classes_count))

    for i in range(len(y)):  # исходит из предположения, что классы будут пронумерованы по порядку, начиная от 0
        transformed[i][y[i]] = 1

    return transformed

In [4]:
y_test_proba = one_hot_transform(y_test, 10)

### Multi-layer Perceptron classifier

In [5]:
clf = MLPClassifier(early_stopping=True).fit(X_train, y_train)  # early stopping чтобы не ждать слишком долго

In [6]:
y_mlp_proba = clf.predict_proba(X_test)
y_mlp = clf.predict(X_test)

### LeNet (Convolutional Neural Network)

LeNet состоит из следующих слоёв:

1. Входной слой: изображение 28x28 пикселей, 1 канал
2. Свёрточный слой: ядро 5x5, padding 2, 6 каналов на выходе, функция активации - гиперболический тангенс
3. AvgPooling: ядро 2x2, stride 2
4. Свёрточный слой: ядро 5x5, 16 каналов на выходе, функция активации - гиперболический тангенс
5. AvgPooling: ядро 2x2, stride 2
6. Уплотнение
7. Плотный слой из 120 нейронов, функция активации - гиперболический тангенс
8. Плотный слой из 84 нейронов, функция активации - гиперболический тангенс
9. Выходной (плотный) слой из 10 нейронов (т.к. цифры от 0 до 9)

Padding нужен для того, чтобы улавливать более тонкие детали - вроде углов картинок. Если бы входное изображение было размером 32x32, то тогда в padding не было бы нужды

In [74]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()

        # Свёрточный слой 1
        # Было изображение 28x28x1, стало 28x28x6, размер не уменьшился т.к. padding добавил к каждой стороне 2

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2) 

        # Свёрточный слой 2
        # Был mapping 14x14x6, стал 10x10x16, как видно, размер уменьшился

        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)

        # AvgPooling
        # Уменьшает размер в 2 раза, т.к. stride = 2 (stride = размерность "куска" для пулинга)
        # Пример: картинку 10x10 просканировали квадратом 2x2 и взяли среднее в этом квадрате, на выходе получится картинка 5x5)

        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

        # Плотный слой 1
        # 400 т.к. после pooling будут мэппинги размером 5x5 и их будет 16 штук, т.е. 5*5*16=400
        
        self.dense1 = nn.Linear(in_features=400, out_features=120) 

        # Плотный слой 2

        self.dense2 = nn.Linear(in_features=120, out_features=84)

        # Плотный слой 3 (выходной)

        self.dense3 = nn.Linear(in_features=84, out_features=10)

        # Функция активации

        self.activation = nn.Tanh()


    def forward(self, x):  # теперь определяем порядок выполнения
        out = self.conv1(x)
        out = self.activation(out)
        
        out = self.pool(out)
        
        out = self.conv2(out)
        out = self.activation(out)
        
        out = self.pool(out)
        out = torch.flatten(out, start_dim=1)  # уплотняем
        
        out = self.dense1(out)
        out = self.activation(out)
        
        out = self.dense2(out)
        out = self.activation(out)
        
        out = self.dense3(out)

        return out

In [75]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # чтобы быстрее обучалось, использую cuda
lenet = LeNet().to(device)

### Преобразование датасета

PyTorch требует, чтобы датасет был определённого формата

In [76]:
# Нормализация

mean_values = X_train.mean(axis=0)
max_min_values = X_train.max(axis=0) - X_train.min(axis=0)

images_train = np.zeros(X_train.shape)
images_test = np.zeros(X_test.shape)

for i in range(X_train.shape[1]):
    if max_min_values[i] != 0:
        images_train[:, i] = (X_train[:, i] - mean_values[i]) / max_min_values[i]

for i in range(X_train.shape[1]):
    if max_min_values[i] != 0:
        images_test[:, i] = (X_test[:, i] - mean_values[i]) / max_min_values[i]

# Преобразования

images_train = images_train.reshape(-1, 1, 28, 28)
images_test = images_test.reshape(-1, 1, 28, 28)

train_dataset = TensorDataset(torch.tensor(images_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
test_dataset = TensorDataset(torch.tensor(images_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))

batch_size = 128

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

### Обучение LeNet

Теперь определяем, как будет обучаться модель. Для этого нужна функция ошибок и оптимизатор (Adam или градиентный спуск)

In [77]:
cost = nn.CrossEntropyLoss()
optimiser = optim.Adam(lenet.parameters(), lr=5*10**-5)

In [78]:
total_epochs = 200

for epoch in range(total_epochs):
    running_loss = 0.0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = lenet(images)
        loss = cost(outputs, labels)
        
        optimiser.zero_grad()  # обнулить градиенты
        loss.backward()
        optimiser.step()  # провести оптимизацию

        running_loss += loss.item()

    print(f"Epoch {epoch+1} out of {total_epochs}, loss is {running_loss}")

Epoch 1 out of 200, loss is 833.4421353340149
Epoch 2 out of 200, loss is 328.73681727051735
Epoch 3 out of 200, loss is 212.82951891422272
Epoch 4 out of 200, loss is 175.90563575923443
Epoch 5 out of 200, loss is 155.26011857390404
Epoch 6 out of 200, loss is 141.89639949798584
Epoch 7 out of 200, loss is 131.6433602720499
Epoch 8 out of 200, loss is 123.98127879202366
Epoch 9 out of 200, loss is 115.70006880164146
Epoch 10 out of 200, loss is 109.27720361948013
Epoch 11 out of 200, loss is 103.00877567380667
Epoch 12 out of 200, loss is 97.57336025685072
Epoch 13 out of 200, loss is 92.24848940968513
Epoch 14 out of 200, loss is 87.46432833373547
Epoch 15 out of 200, loss is 82.76759923249483
Epoch 16 out of 200, loss is 78.67767161875963
Epoch 17 out of 200, loss is 74.62776070833206
Epoch 18 out of 200, loss is 71.04065188765526
Epoch 19 out of 200, loss is 67.46695790998638
Epoch 20 out of 200, loss is 64.40154929086566
Epoch 21 out of 200, loss is 61.272392980754375
Epoch 22 out

KeyboardInterrupt: 

### Предсказание LeNet

In [ ]:
lenet.eval()  # переводим в режим оценки

y_net_proba = []
y_net =[]

with torch.no_grad():  # уменьшить потребление памяти
    for images, labels in test_loader:
        images = images.to(device)
        outputs = lenet(images)

        for proba in outputs.cpu().numpy():  # вероятности
            y_net_proba.append(proba)

        for label in outputs.argmax(dim=1).cpu().numpy():  # предсказанные классы
            y_net.append(label)

y_net_proba = np.array(y_net_proba)
y_net = np.array(y_net)

### Метрики MLP

In [ ]:
accuracy_score(y_test, y_mlp)

In [ ]:
recall_score(y_test, y_mlp, average="macro")

In [ ]:
precision_score(y_test, y_mlp, average="macro")

In [ ]:
f1_score(y_test, y_mlp, average="macro")

In [ ]:
roc_auc_score(y_test_proba, y_mlp_proba)

### Метрики LeNet

In [ ]:
accuracy_score(y_test, y_net)

In [ ]:
recall_score(y_test, y_net, average="macro")

In [ ]:
precision_score(y_test, y_net, average="macro")

In [ ]:
f1_score(y_test, y_net, average="macro")

In [ ]:
roc_auc_score(y_test_proba, y_net_proba)

### Вывод